In [1]:
import numpy as np
import pandas as pd

import time

import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing import image as tfk_image
from tensorflow.keras.preprocessing.image import ImageDataGenerator as tfk_ImageDataGenerator
from tensorflow.python.keras.applications.efficientnet import EfficientNetB7 as tfk_BaseModel
from tensorflow.python.keras.applications.efficientnet import preprocess_input as tfk_preprocess_input

from PIL import Image, ImageEnhance

import matplotlib.pyplot as plt
import seaborn as sns; sns.set();

from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, log_loss, accuracy_score
from sklearn.preprocessing import OneHotEncoder

## Load dataset

In [4]:
pd_test = pd.read_csv('shopee-product-detection-dataset/test.csv')
pd_test.head()

,filename,category
0,fd663cf2b6e1d7b02938c6aaae0a32d2.jpg,43
1,c7fd77508a8c355eaab0d4e10efd6b15.jpg,43
2,127f3e6d6e3491b2459812353f33a913.jpg,43
3,5ca4f2da11eda083064e6c36f37eeb81.jpg,43
4,46d681a542f2c71be017eef6aae23313.jpg,43


In [5]:
batch_size=1
input_shape=(224, 224, 3)

In [6]:
test_datagen = tfk_ImageDataGenerator(
    preprocessing_function=tfk_preprocess_input
)

test_generator = test_datagen.flow_from_dataframe(
    pd_test,
    directory='shopee-product-detection-dataset/test/',
    x_col='filename',
    target_size=input_shape[:2],
    color_mode='rgb',
    batch_size=batch_size,
    class_mode=None,
    shuffle=False,
)

Found 12186 validated image filenames.


## Load model

In [9]:
with open('gcloud-models/dict.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

interpreter = tf.lite.Interpreter(model_path="gcloud-models/model.tflite")

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [10]:
interpreter.resize_tensor_input(input_details[0]['index'], [batch_size, *input_shape])
interpreter.resize_tensor_input(output_details[0]['index'],[batch_size, *input_shape])

interpreter.allocate_tensors()

## Test

In [ ]:
%%time
y_pred = []

num_steps = test_generator.n//batch_size+1
print("num steps:", num_steps)

start = time.time()
for i in range(num_steps):
  if (i+1) % 100 == 0:
    print("step:", i+1, time.time() - start)
    start = time.time()
  
  x = next(iter(test_generator))
  x = x.astype(np.uint8)
  
  interpreter.set_tensor(input_details[0]['index'], x)
  interpreter.invoke()
  output_data = interpreter.get_tensor(output_details[0]['index'])
  output_labels = np.asarray([int(classes[i]) for i in output_data.argmax(axis=1)])
  y_pred.extend(output_labels)

y_pred = np.array(y_pred)

num steps: 12187
step: 100 33.054864168167114
step: 200 33.55562114715576
step: 300 32.36565017700195
step: 400 31.918362140655518
step: 500 31.835959672927856
step: 600 32.1205530166626
step: 700 32.15298128128052
step: 800 34.4155809879303
step: 900 31.622830152511597
step: 1000 31.811530113220215
step: 1100 32.67298245429993
step: 1200 34.17414712905884
step: 1300 32.8804292678833
step: 1400 31.9122257232666
step: 1500 32.133999824523926
step: 1600 32.498602628707886
step: 1700 33.042184591293335
step: 1800 33.724451303482056
step: 1900 34.17407464981079
step: 2000 33.19287347793579
step: 2100 32.47928190231323
step: 2200 32.043450355529785
step: 2300 33.33682203292847
step: 2400 32.96821618080139
step: 2500 32.50149607658386
step: 2600 32.5614857673645
step: 2700 32.217424154281616
step: 2800 32.50444006919861
step: 2900 31.78349280357361
step: 3000 33.27624154090881
step: 3100 33.207152366638184
step: 3200 31.9429931640625
step: 3300 32.05816698074341
step: 3400 32.06964111328125
